## Модель Word2vec.
### Выполнил: Хорин М. А.

Перед обучением модели Word2vec необходимо найти коллекцию текстов для её обучения. Я использовал тексты, предложенные в ДЗ из https://www.kaggle.com/c/word2vec-nlp-tutorial/data. Тексты находятся в нескольких файлах формата .tsv и являются обзорами на фильмы из базы данных о кинематографе IMDB.<br>
<br>Считаем данные из каждого файла, используя pandas, и соберем одну большую коллекцию отзывов.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

'''
считываем данные из файлов с обзорами фильмов 
header=0 означает, что первая строка файла содержит заголовки
delimeter=\t означает, что разделителем в файлах является табуляция \t
quoting=3 означает игнорировать двойные кавычки при считывании файла
'''
df1 = pd.read_csv('unlabeledTrainData.tsv', delimiter='\t', quoting=3, header=0)
df2 = pd.read_csv('labeledTrainData.tsv', delimiter='\t', quoting=3, header=0)
df3 = pd.read_csv('testData.tsv', delimiter='\t', quoting=3, header=0)

df = df1.append(df2.append(df3)) # соединяем все обзоры вместе
print('Всего обзоров:', df.shape[0])

Всего обзоров: 100000


Как можно заметить выше, собрана довольно внушительная коллекция текстов, насчитывающая сто тысяч обзоров.<br><br>Взглянем на некоторые из них.

In [2]:
collection = list(df.review.values)
collection[:2]

['"Watching Time Chasers, it obvious that it was made by a bunch of friends. Maybe they were sitting around one day in film school and said, \\"Hey, let\'s pool our money together and make a really bad movie!\\" Or something like that. What ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. All corners were cut, except the one that would have prevented this film\'s release. Life\'s like that."',
 '"I saw this film about 20 years ago and remember it as being particularly nasty. I believe it is based on a true incident: a young man breaks into a nurses\' home and rapes, tortures and kills various women.<br /><br />It is in black and white but saves the colour for one shocking shot.<br /><br />At the end the film seems to be trying to make some political statement but it just comes across as confused and obscene.<br /><br />Avoid."']

Как можно заметить, исходные текстовые данные малопригодны для обработки, поскольку содержат знаки пунктуации, символы кодировки, html теги и иной ненужный мусор, от которого необходимо избавиться.<br><br>
Поскольку на вход модели word2vec необходимо подавать список предложений (другими словами, список списков), реализуем две функции. Функция make_sentences будет разбивать текст обзора на предложения по знакам пунктуации, используя токенизатор nltk, а также создавать список предложений обзора, очищенных от ненужного мусора второй функцией make_wordlist.

In [3]:
from bs4 import BeautifulSoup
import re

"""
функция очищает одно предложение обзора от мусора
библиотека BeautifulSoup чистит текст от HTML тегов
очистка от знаков пунктуации проводится регулярными выражениями
"""
def make_wordlist(sentence):
    text = BeautifulSoup(sentence).get_text() # чистим текст от тегов HTML
    text = re.sub("[^a-zA-Z]"," ", text) # чистим текст от знаков пунктуации
    return text.lower().split() # вовзращаем текст предложения в виде списка слов в нижнем регистре

In [4]:
# инициируем токенизатор
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

"""
функция преобразует цельный текст обзора в список списков
"""
def make_sentences(review, tokenizer):
    review_sentences = tokenizer.tokenize(review.strip()) # обрезаем текст обзора от пробелов в начале и конце,
                                                          # получаем список предложений обзора
    
    sentences = [] # создаём переменную для хранения предложений обзора в виде списков слов
    
    # чистим и преобразуем каждое предложение обзора в список слов и добавляем в список sentences
    for sentence in review_sentences:
        if len(sentence) > 0: # проверка на пустое предложение
            sentences.append(make_wordlist(sentence))
    return sentences

Создадим список предложений из нашей коллекции, используя две вышереализованные функции.

In [5]:
%%time
w2v_data = []

for review in collection:
    w2v_data += make_sentences(review, tokenizer)

Wall time: 7min 21s


In [6]:
print(w2v_data[0:3])

[['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends'], ['maybe', 'they', 'were', 'sitting', 'around', 'one', 'day', 'in', 'film', 'school', 'and', 'said', 'hey', 'let', 's', 'pool', 'our', 'money', 'together', 'and', 'make', 'a', 'really', 'bad', 'movie', 'or', 'something', 'like', 'that'], ['what', 'ever', 'they', 'said', 'they', 'still', 'ended', 'up', 'making', 'a', 'really', 'bad', 'movie', 'dull', 'story', 'bad', 'script', 'lame', 'acting', 'poor', 'cinematography', 'bottom', 'of', 'the', 'barrel', 'stock', 'music', 'etc']]


In [7]:
print('Количество предложений:', len(w2v_data))

Количество предложений: 1056938


Как можно заметить, теперь исходная коллекция текстов пригодна для того, чтобы обучать модель word2vec. Её размер составил более миллиона предложений, что довольно-таки неплохо.
<br><br>
<b>1.Обучим модель word2vec. Оценим время обучения модели,используя модуль time.</b><br><br>
Word2vec повзоляет настраивать параметры, влияющие на качество модели. Изучив tutorial, рекомендуемый в документации к word2vec библиотеки gensim (https://rare-technologies.com/word2vec-tutorial/), мною были выбраны следующие значения параметров:
* нейросетевая архитектура - CBOW;
* workers - 4;
* size - 300; 
* min_count - 20;
* window (контекстное окно) - 5;
* sample - 1e-3.<br><br>
Остальные параметры по умолчанию.

In [8]:
from gensim.models import word2vec
import time

beginning = time.clock() # засекаем начало обучения

model = word2vec.Word2Vec(w2v_data, workers=4, min_count=20, size=300, window=5, sample=1e-3) # обучаем модель

print('Время обучения, сек:', round(time.clock()-beginning, 2))

Время обучения, сек: 206.86


<b>2. Приведём 5-10 примеров использования .most_similar для определения близких слов.</b>

In [9]:
model.most_similar('nice', topn=5)

[('neat', 0.6948410272598267),
 ('cool', 0.6063452959060669),
 ('good', 0.6040568351745605),
 ('pleasant', 0.5974904298782349),
 ('refreshing', 0.5724823474884033)]

In [10]:
model.most_similar('actor', topn=5)

[('actress', 0.6907398700714111),
 ('performer', 0.6144192218780518),
 ('comedian', 0.6075437068939209),
 ('actors', 0.5465003848075867),
 ('role', 0.5140215754508972)]

In [11]:
model.most_similar('awful', topn=5)

[('terrible', 0.7992217540740967),
 ('atrocious', 0.7668658494949341),
 ('horrible', 0.7535910606384277),
 ('dreadful', 0.7530136704444885),
 ('abysmal', 0.7430434226989746)]

In [12]:
model.most_similar('film', topn=5)

[('movie', 0.9037315845489502),
 ('flick', 0.6656942367553711),
 ('documentary', 0.6437262892723083),
 ('picture', 0.6157901287078857),
 ('films', 0.5460703372955322)]

In [13]:
model.most_similar('soccer', topn=5)

[('football', 0.8191125988960266),
 ('basketball', 0.7687404155731201),
 ('poker', 0.7153268456459045),
 ('baseball', 0.6736490726470947),
 ('tennis', 0.6441650390625)]

In [15]:
model.most_similar('anger', topn=5)

[('frustration', 0.8575396537780762),
 ('despair', 0.7271488308906555),
 ('sadness', 0.7240976691246033),
 ('bitterness', 0.7201728224754333),
 ('anguish', 0.6986355185508728)]

Как можно заметить, всего для определения близости слов были использованы 6 примеров. В итоге, модель правильно и довольно качественно находит слова близкие по значению к исходным. В большинстве случаев найденные слова не являются синонимами исходных, но могут употребляться как их слова-заменители или использоваться в похожей области упортребления. Тем не менее, многие из найденных слов семантически близки к используемым 7 примерам, некоторые из них также являются синонимами исходных слов.<br><br>
Таким образом, для данного небольшого корпуса текстов модель работает на довольно высоком уровне. Работу модели можно улучшить, добавив больше текстов для обучения и произведя более тщательную настройку параметров.

<b>3. Приведём 5-10 примеров использования .most_similar для определения ассоциаций (А к Б, как В к ?). </b>

In [16]:
model.most_similar(['boy', 'woman'], ['man'], 3)

[('girl', 0.7545726299285889),
 ('lad', 0.5541477203369141),
 ('prostitute', 0.5534797310829163)]

Ассоциация определена верно, так как "man" к "boy", как "woman" к "girl".

In [17]:
model.most_similar(['better', 'bad'], ['good'], 3)

[('worse', 0.6495989561080933),
 ('funnier', 0.5254871249198914),
 ('cheaper', 0.4639887809753418)]

Ассоциация определена верно, так как "good" к "better", как "bad" к "worse".

In [18]:
model.most_similar(['daughter', 'father'], ['mother'], 3)

[('son', 0.7639557123184204),
 ('niece', 0.7021713256835938),
 ('grandson', 0.6820499897003174)]

Ассоциация определена верно, так как "mother" к "daughter", как "father" к "son".

In [19]:
model.most_similar(['actor', 'woman'], ['man'], 3)

[('actress', 0.7081006765365601),
 ('performer', 0.5143195390701294),
 ('dancer', 0.45161423087120056)]

Ассоциация определена верно, так как "man" к "actor", как "woman" к "actress".

In [20]:
model.most_similar(['musician', 'art'], ['music'], 3)

[('artist', 0.5958595871925354),
 ('painter', 0.550783634185791),
 ('visionary', 0.5077059268951416)]

Ассоциация определена верно, так как "music" к "musician", как "art" к "artist".<br><br>
Таким образом, используемая модель корректно находит ассоциации между словами. Тем не менее, лишь первые слова из .most_similar являются верными для ассоциаций. Посмотрим, насколько качественно модель определяет лишние слова.

<b>4. Приведем 5-10 примеров использования .doesnt_match для определения лишнего слова.</b>

In [21]:
model.doesnt_match('china russia spain italy london'.split())

'london'

Лишнее слово определено верно, так как london является городом, а не страной.

In [22]:
model.doesnt_match('day month year hour'.split())

'hour'

Лишнее слово определено верно, так как hour не является элементом даты, в отличие от day, month и year.

In [28]:
model.doesnt_match('monkey tiger wolf fish'.split())

'fish'

Лишнее слово определено верно, так как fish не является млекопитающим.

In [29]:
model.doesnt_match('bob tommy alan mike mary'.split())

'mary'

Лишнее слово определено верно, так как mary не является мужским именем.

In [30]:
model.doesnt_match('weapon bullets fire love'.split())

'love'

Лишнее слово определено верно, так как love не имеет отношения к понятиям, связанным с войной.

In [31]:
model.doesnt_match('art literature cinema science'.split())

'science'

Лишнее слово определено верно, так как science не является видом искусства.<br><br>
Таким образом,  во всех рассмотренных 6 примерах модель корректно нашла лишние слова, что означает верную настройку параметров и довольно высокое качество модели.

<b>5. Попробуем найти такие пары и тройки слов для которых:</b>
* <b>не выполняются свойства коммутативности и транзитивности относительно операции определения близких слов. </b>

Коммутативность означает, что если слово X входит в топ 3 по .most_similar для слова Y, то слово Y также входит в топ три по .most_similar для слова X.<br><br>
Транзитивность означает, что если слово X входит в топ 3 по .most_similar для слова Y, а слово Y входит в топ три по .most_similar для слова Z, то слово X входит в топ 3 по .most_similar для слова Z.<br><br>
Начнем с проверки коммутативности для пар слов.<br> Первая пара:

In [32]:
model.most_similar('man', topn=3)

[('woman', 0.6300959587097168),
 ('lad', 0.6169289946556091),
 ('lady', 0.5826666355133057)]

woman входит в топ 3 по .most_similar для man. Посмотрим, выполняется ли обратное.

In [33]:
model.most_similar('woman', topn=3)

[('prostitute', 0.6766529083251953),
 ('girl', 0.6726246476173401),
 ('lady', 0.6719728708267212)]

Как можно заметить, обратное не выполняется. Следовательно, свойство коммутативности для пары (woman, man) не выполняется.<br><br>Вторая пара:

In [34]:
model.most_similar('paris', topn=3)

[('france', 0.6665081977844238),
 ('thailand', 0.6633102297782898),
 ('london', 0.6543998122215271)]

france входит в топ 3 по .most_similar для paris. Посмотрим, выполняется ли обратное.

In [35]:
model.most_similar('france', topn=3)

[('spain', 0.834465742111206),
 ('italy', 0.8278445601463318),
 ('germany', 0.8073477745056152)]

Как можно заметить, обратное не выполняется. Следовательно, свойство коммутативности для пары (france, paris) не выполняется.<br><br>Третья пара:

In [36]:
model.most_similar('actor', topn=3)

[('actress', 0.6907398700714111),
 ('performer', 0.6144192218780518),
 ('comedian', 0.6075437068939209)]

performer входит в топ 3 по .most_similar для actor. Посмотрим, выполняется ли обратное.

In [37]:
model.most_similar('performer', topn=3)

[('dancer', 0.7036668658256531),
 ('comedienne', 0.695297122001648),
 ('pianist', 0.6823179721832275)]

Как можно заметить, обратное не выполняется. Следовательно, свойство коммутативности для пары (performer, actor) не выполняется.<br><br>
Проверим свойство транзитивности для троек слов.<br> Первая тройка:

In [38]:
model.most_similar('woman', topn=3)

[('prostitute', 0.6766529083251953),
 ('girl', 0.6726246476173401),
 ('lady', 0.6719728708267212)]

In [39]:
model.most_similar('lady', topn=3)

[('woman', 0.6719728708267212),
 ('maid', 0.643402636051178),
 ('widow', 0.6131343841552734)]

girl входит в топ 3 по .most_similar для woman, woman входит в топ 3 по .most_similar для lady, однако girl не входит в топ 3 по .most_similar для lady. Следовательно, для тройки (girl, woman, lady) свойство транзитивности не выполняется.<br><br>
Вторая тройка:

In [42]:
model.most_similar('poland', topn=3)

[('spain', 0.8311604857444763),
 ('greece', 0.8245682120323181),
 ('italy', 0.7992098927497864)]

In [43]:
model.most_similar('italy', topn=3)

[('spain', 0.849463939666748),
 ('france', 0.8278445601463318),
 ('poland', 0.7992098927497864)]

greece входит в топ 3 по .most_similar для poland, poland входит в топ 3 по .most_similar для italy, однако greece не входит в топ 3 по .most_similar для italy. Следовательно, для тройки (greece, poland, italy) свойство транзитивности не выполняется.<br><br>
Третья тройка:

In [44]:
model.most_similar('movie', topn=3)

[('film', 0.903731644153595),
 ('flick', 0.735821008682251),
 ('it', 0.5866470336914062)]

In [45]:
model.most_similar('picture', topn=3)

[('film', 0.6157901883125305),
 ('movie', 0.5819380879402161),
 ('pictures', 0.5416737794876099)]

flick входит в топ 3 по .most_similar для movie, movie входит в топ 3 по .most_similar для picture, однако flick не входит в топ 3 по .most_similar для picture. Следовательно, для тройки (flick, movie, picture) свойство транзитивности не выполняется.

* <b>выполняются свойства коммутативности и транзитивности относительно операции определения близких слов.<b>

Начнем с проверки <i>коммутативности</i> для пар слов.<br> <i>Первая пара:

In [46]:
model.most_similar('movie', topn=3)

[('film', 0.903731644153595),
 ('flick', 0.735821008682251),
 ('it', 0.5866470336914062)]

film входит в топ 3 по .most_similar для movie. Посмотрим, выполняется ли обратное.

In [47]:
model.most_similar('film', topn=3)

[('movie', 0.9037315845489502),
 ('flick', 0.6656942367553711),
 ('documentary', 0.6437262892723083)]

Как можно заметить, обратное выполняется. Следовательно, свойство коммутативности для пары (film, movie) выполняется.<br><br><i>Вторая пара:

In [48]:
model.most_similar('female', topn=3)

[('male', 0.7954157590866089),
 ('chauvinist', 0.6137508153915405),
 ('heterosexual', 0.5550321340560913)]

male входит в топ 3 по .most_similar для female. Посмотрим, выполняется ли обратное.

In [49]:
model.most_similar('male', topn=3)

[('female', 0.7954157590866089),
 ('heterosexual', 0.6558157801628113),
 ('chauvinist', 0.6060659289360046)]

Как можно заметить, обратное выполняется. Следовательно, свойство коммутативности для пары (male, female) выполняется.<br><br><i>Третья пара:

In [50]:
model.most_similar('actress', topn=3)

[('actor', 0.6907398700714111),
 ('performer', 0.6499398946762085),
 ('comedienne', 0.6012341976165771)]

actor входит в топ 3 по .most_similar для actress. Посмотрим, выполняется ли обратное.

In [51]:
model.most_similar('actor', topn=3)

[('actress', 0.6907398700714111),
 ('performer', 0.6144192218780518),
 ('comedian', 0.6075437068939209)]

Как можно заметить, обратное выполняется. Следовательно, свойство коммутативности для пары (actor, actress) выполняется.<br><br>
Проверим свойство <i>транзитивности</i> для троек слов.<br> <i>Первая тройка:

In [52]:
model.most_similar('film', topn=3)

[('movie', 0.9037315845489502),
 ('flick', 0.6656942367553711),
 ('documentary', 0.6437262892723083)]

In [53]:
model.most_similar('documentary', topn=3)

[('film', 0.6437263488769531),
 ('movie', 0.5828584432601929),
 ('biopic', 0.5619769096374512)]

movie входит в топ 3 по .most_similar для film, film входит в топ 3 по .most_similar для documentary, movie также входит в топ 3 по .most_similar для documentary. Следовательно, для тройки (movie, film, documentary) свойство транзитивности выполняется.<br><br>
<i>Вторая тройка:

In [78]:
model.most_similar('actor', topn=3)

[('actress', 0.6907398700714111),
 ('performer', 0.6144192218780518),
 ('comedian', 0.6075437068939209)]

In [79]:
model.most_similar('comedian', topn=3)

[('performer', 0.6665052771568298),
 ('rowan', 0.6112197637557983),
 ('actor', 0.6075436472892761)]

performer входит в топ 3 по .most_similar для actor, actor входит в топ 3 по .most_similar для comedian, performer также входит в топ 3 по .most_similar для comedian. Следовательно, для тройки (performer, actor, comedian) свойство транзитивности выполняется.<br><br>
<i>Третья тройка:

In [81]:
model.most_similar('spain', topn=3)

[('italy', 0.849463939666748),
 ('france', 0.834465742111206),
 ('poland', 0.8311604261398315)]

In [208]:
model.most_similar('italy', topn=3)

[('spain', 0.8480809926986694),
 ('france', 0.8153465986251831),
 ('germany', 0.8083160519599915)]

france входит в топ 3 по .most_similar для spain, spain входит в топ 3 по .most_similar для italy, france также входит в топ 3 по .most_similar для italy. Следовательно, для тройки (france, spain, italy) свойство транзитивности выполняется.

#### Вывод: 
Таким образом, word2vec представляет из себя мощный инструмент для расчёта векторных представлений слов и анализа семантики естественных языков. Собрав достаточно большую коллекцию текстов и предварительно её обработав, а также подобрав оптимальные параметры, удалось обучить модель word2vec довольно высокого качества. Использование обученной модели позволяет корректно определять семантически близкие слова, правильно находить ассоциации между словами, определять лишние слова в контексте. Также удалось найти пары и тройки слов, для которых как выполняются, так и не выполняются свойства коммутативности и транзитивности.